In [ ]:
import pandas as pd
from IPython.display import display

data_bestand = "../data/red-wine.csv"
data_rode_wijn = pd.read_csv(data_bestand)

# Maak een training en controle splits
df_train = data_rode_wijn.sample(frac=0.7, random_state=0)
df_valid = data_rode_wijn.drop(df_train.index)
display(df_train.head(4))

# Scale tot [0, 1]
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)
df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

# Split kenmerken en doel
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimale hoeveelheid die we een verbetering noemen
    patience=20, # hoeveel epochs voor we stoppen
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])
model.compile(
    optimizer='adam',
    loss='mae',
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping], # plaats de callbacks in een list
    verbose=0,  # schakel de training log uit
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimale controle verlies: {}".format(history_df['val_loss'].min()))